In [1]:
import bw2data as bd
import bw2calc as bc
import bw2analyzer as ba

In [2]:
bd.projects.set_current("ei 3.9 cutoff")

In [3]:
ht = (
    'EF v3.1 no LT',
    'human toxicity: carcinogenic no LT',
    'comparative toxic unit for human (CTUh) no LT'
)

In [4]:
act = bd.get_activity(name="market for wafer, fabricated, for integrated circuit")

In [5]:
lca = bc.LCA({act: 1}, ht, use_distributions=True)
lca.lci()
lca.lcia()

/Users/chrismutel/miniconda3/envs/bw25/lib/python3.10/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 3.73e+13)
  warnings.warn(msg, UmfpackWarning)


In [6]:
while lca.score >= 0:
    next(lca)

In [7]:
lca.to_dataframe(matrix_label="characterized_inventory", cutoff=10)

,row_index,col_index,amount,row_id,col_id,row_database,row_code,row_name,row_location,row_unit,row_type,row_categories,row_product,col_database,col_code,col_name,col_location,col_unit,col_type,col_reference_product
0,1016,12309,-0.002499,2175,17019,biosphere3,e429b852-e421-4fcb-8a9b-b0241863bfb2,Cadmium II,None,kilogram,emission,soil::agricultural,None,ecoinvent-3.9-cutoff,4712961cceee83b212c1ab360d922180,sugarcane production,BR-SP,kilogram,process,sugarcane
1,1027,12309,-0.001250,2189,17019,biosphere3,e7881581-21b3-4f5c-bd63-6b0684b5e712,Chromium III,None,kilogram,emission,soil::agricultural,None,ecoinvent-3.9-cutoff,4712961cceee83b212c1ab360d922180,sugarcane production,BR-SP,kilogram,process,sugarcane
2,2080,12309,0.000740,4290,17019,biosphere3,4673a799-36a5-4359-ac6a-fa933281bc93,Arsenic ion,None,kilogram,emission,soil::agricultural,None,ecoinvent-3.9-cutoff,4712961cceee83b212c1ab360d922180,sugarcane production,BR-SP,kilogram,process,sugarcane
3,1048,12309,-0.000348,2220,17019,biosphere3,4010918f-7fd0-4925-8fbb-8fd7a44a806c,Lead II,None,kilogram,emission,soil::agricultural,None,ecoinvent-3.9-cutoff,4712961cceee83b212c1ab360d922180,sugarcane production,BR-SP,kilogram,process,sugarcane
4,1016,18691,-0.000770,2175,23401,biosphere3,e429b852-e421-4fcb-8a9b-b0241863bfb2,Cadmium II,None,kilogram,emission,soil::agricultural,None,ecoinvent-3.9-cutoff,c4c8962a99c8166a61c4b4712c339b0f,sugarcane production,BR-MG,kilogram,process,sugarcane
5,1027,18691,-0.000297,2189,23401,biosphere3,e7881581-21b3-4f5c-bd63-6b0684b5e712,Chromium III,None,kilogram,emission,soil::agricultural,None,ecoinvent-3.9-cutoff,c4c8962a99c8166a61c4b4712c339b0f,sugarcane production,BR-MG,kilogram,process,sugarcane
6,1016,2114,-0.000329,2175,6824,biosphere3,e429b852-e421-4fcb-8a9b-b0241863bfb2,Cadmium II,None,kilogram,emission,soil::agricultural,None,ecoinvent-3.9-cutoff,97518d3954f9a7dc83f505d5deb1e345,sugarcane production,BR-PR,kilogram,process,sugarcane
7,1016,19093,0.000228,2175,23803,biosphere3,e429b852-e421-4fcb-8a9b-b0241863bfb2,Cadmium II,None,kilogram,emission,soil::agricultural,None,ecoinvent-3.9-cutoff,29db7abce67e87309bfc6671574a4ef1,sugarcane production,BR-MS,kilogram,process,sugarcane
8,1027,6965,0.000239,2189,11675,biosphere3,e7881581-21b3-4f5c-bd63-6b0684b5e712,Chromium III,None,kilogram,emission,soil::agricultural,None,ecoinvent-3.9-cutoff,ed52ccba49a478e3533fa4ac8f5a8710,sugarcane production,BR-GO,kilogram,process,sugarcane
9,2080,6965,-0.000203,4290,11675,biosphere3,4673a799-36a5-4359-ac6a-fa933281bc93,Arsenic ion,None,kilogram,emission,soil::agricultural,None,ecoinvent-3.9-cutoff,ed52ccba49a478e3533fa4ac8f5a8710,sugarcane production,BR-GO,kilogram,process,sugarcane


Look up exchange value for first row in inventory matrix

In [8]:
row = 1016
col = 12309

In [9]:
lca.inventory[row, col]

-27.762874920670892

Negative value from biosphere or technosphere?

In [10]:
lca.biosphere_matrix[row, col]

-1.413550491016362

Get information on the uncertainty distribution of this point.

In [11]:
mask = (
    (lca.biosphere_mm.input_row_col_indices()['row'] == row) *
    (lca.biosphere_mm.input_row_col_indices()['col'] == col)
)

This is [normally distributed](https://stats-arrays.readthedocs.io/en/latest/#mapping-parameter-array-columns-to-uncertainty-distributions), so can be positive or negative. Look how big $\sigma$ is versus $\mu$.

In [12]:
lca.biosphere_mm.input_uncertainties()[mask]

array([(3, -2.2214245e-08, 1.1916375, nan, nan, nan, False)],
      dtype=[('uncertainty_type', 'u1'), ('loc', '<f4'), ('scale', '<f4'), ('shape', '<f4'), ('minimum', '<f4'), ('maximum', '<f4'), ('negative', '?')])

How does sugarcane get into our wafer production?

In [13]:
ba.print_recursive_calculation(
    activity=act,
    lcia_method=ht,
    max_level=8,
    cutoff=0.1,
    use_matrix_values=True,
    _lca_obj=lca,
    _total_score=float(lca.score)
)

/Users/chrismutel/miniconda3/envs/bw25/lib/python3.10/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 3.73e+13)
  warnings.warn(msg, UmfpackWarning)


Fraction of score | Absolute score | Amount | Activity
0001 | -0.004895 |     1 | 'market for wafer, fabricated, for integrated circuit' (square meter, 
  0001 | -0.004895 |     1 | 'wafer production, fabricated, for integrated circuit' (square meter, 
    0.889 | -0.004351 | 2.76e+04 | 'market group for electricity, medium voltage' (kilowatt hour, GLO, No
      0.881 | -0.004313 |  1627 | 'market group for electricity, medium voltage' (kilowatt hour, RLA, No
        0.881 | -0.004311 | 699.6 | 'market group for electricity, medium voltage' (kilowatt hour, BR, Non
          0.214 | -0.001048 |   181 | 'market for electricity, medium voltage' (kilowatt hour, BR-Southern g
            0.214 | -0.001048 | 182.4 | 'electricity voltage transformation from high to medium voltage' (kilo
              0.214 | -0.001048 | 184.7 | 'market for electricity, high voltage' (kilowatt hour, BR-Southern gri
          0.605 | -0.00296 | 311.9 | 'market for electricity, medium voltage' (kilowatt hour, BR